In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install --upgrade agno
# !{sys.executable} -m pip install --upgrade yfinance

In [4]:
from agno.tools import Toolkit
from datetime import datetime, timedelta
import yfinance as yf

class StockPriceTool(Toolkit):
    def __init__(self):
        super().__init__(name='stock_price_toolkit')
        self.register(self.date_range)
        self.register(self.stock_price)

    def date_range(self, user_date:str):
        """Calculates the start and end dates for a given date.
        Args:
            user_date (str): The date provided by the user to work out the range.
        Returns:
            str: A message indicating the start and end date ranges
        """        
        # Define the target date from the user's prompt
        target_date = datetime.strptime(user_date, '%Y-%m-%d').date()
        
        # Define today's date
        today = datetime.now().date()
        
        # Calculate the difference in days
        delta = (today - target_date).days
        
        # Apply the rules for the end date
        if delta <= 0:
            end_date = today + timedelta(days=1)
        elif delta > 3:
            end_date = target_date + timedelta(days=3)
        else:
            end_date = today
        
        # Calculate the start date based on the determined end date
        start_date = end_date - timedelta(days=6)
        return f'Start date is {start_date} and the end date is {end_date}'
        
    def stock_price(self, ticker:str, target_date:str, start_date:str, end_date:str) -> str:
        """Retrieves the current stock price for the given ticker symbol withing given date range.
        Args:
            ticker (str): The stock ticker symbol (e.g., 'AAPL', 'TSLA').
            target_date (str): The date we want to get the price for.
            start_date (str): The start date of the date range.
            end_date (str): The end date of the date range.
        Returns:
            str: A message indicating the current stock price, or an error message if the ticker is invalid.
        """
        try:
            # Get the prices from yfinance
            data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)['Close']
            if not data.empty:
                # Get the nearest index as the target date may not exist
                pos = data.index.get_indexer([target_date], method='nearest')[0]
                price = data.iloc[pos, 0]
                actual_date = data.index[pos].strftime('%Y-%m-%d')
                if target_date != actual_date:
                    return f'The stock price of {ticker} on the closest day {actual_date} is ${price:.2f}'
                else:
                    # Found the price on the target date
                    return f'The stock price of {ticker} on {target_date} is ${price:.2f}.'
            else:
                return f'Could not find current stock price for {ticker}.' 
        except Exception as e:
            return f'Error retrieving stock price for {ticker}: {str(e)}'

In [5]:
from agno.agent import Agent
from agno.models.groq import Groq

instructions=[
    'Get the start and end dates using the target date first before getting the stock price',
    'Indicate if the stock price date is different to the target date'
]
agent = Agent(
    name='Stock Analyst',
    role='You are a helpful Assistant to get stock prices using tools',
    model=Groq(id='llama3-8b-8192'),
    instructions=instructions,
    tools=[StockPriceTool()],
    show_tool_calls=True,
    # debug_mode=True
)

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from agno.utils.pprint import pprint_run_response

target_date = '2025-08-08'
ticker = 'GOOG'
prompt = f"What's the stock price of ticker {ticker} on {target_date}?"
response = agent.run(prompt)
pprint_run_response(response)
# print(response.content)

[*********************100%***********************]  1 of 1 completed


╭───────────────────────────────────────────────────╮
│ The stock price of GOOG on 2025-08-08 is $202.09. │
╰───────────────────────────────────────────────────╯